In [ ]:
!pip install pandas openpyxl mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\kira\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import numpy as np


In [ ]:
# ---------- CONFIG ----------
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "1234",
    "database": "bank"}

In [ ]:
file_path = "customers_100.xlsx"
table_name = "customers"
expected_cols = ["CustomerID", "FullName", "Email", "Phone", "Address", "DateOfBirth"]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Ensure expected columns exist (try to auto-rename common variants)
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}. Rename them to {expected_cols} or tell me the actual names.")

# 3) Reorder/select the expected columns
df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols  # standardize names
print("Preview (first 5 rows):")
display(df.head())

# 4) Clean and convert types
df["CustomerID"] = pd.to_numeric(df["CustomerID"], errors="coerce").astype("Int64")
df["DateOfBirth"] = pd.to_datetime(df["DateOfBirth"], dayfirst=True, errors="coerce")

# Format date strings as MySQL expects
df["DateOfBirth"] = df["DateOfBirth"].dt.strftime("%Y-%m-%d")

# Replace NaN with None for DB insertion
df["DateOfBirth"] = df["DateOfBirth"].where(df["DateOfBirth"].notnull(), None)
df = df.replace({np.nan: None})


df["Address"] = df["Address"].apply(lambda x: x[:50] if isinstance(x, str) else x)

# 5) Prepare data tuples in correct order
records = [tuple(x) for x in df[expected_cols].to_numpy()]

# Quick sanity checks
null_keys = [r for r in records if r[0] is None]
if null_keys:
    print(f"Warning: {len(null_keys)} rows have NULL CustomerID (primary key) — these will likely fail.")
    display(null_keys[:3])

print("Total prepared rows to insert:", len(records))

# 6) Connect to MySQL and insert
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    print("Connected to MySQL:", db_config["host"], "DB:", db_config["database"])

    cursor.execute(f"DESCRIBE {table_name};")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (CustomerID, FullName, Email, Phone, Address, Date_Of_Birth)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
      FullName = VALUES(FullName),
      Email = VALUES(Email),
      Phone = VALUES(Phone),
      Address = VALUES(Address),
      Date_Of_Birth = VALUES(Date_Of_Birth)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed with error:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


Loaded rows: 100
Columns in file: ['CustomerID', 'FullName', 'Email', 'Phone', 'Address', 'DateOfBirth']
Preview (first 5 rows):



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\kira\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


,CustomerID,FullName,Email,Phone,Address,DateOfBirth
0,1,Allison Hill,jillrhodes@miller.com,2.016866e+11,"386 Shane Harbors, Port Lindachester, KY 20880",1969-04-02
1,2,Gabrielle Davis,melanie94@blair.com,NaN,"Unit 6184 Box 9593, DPO AP 09617",1957-09-19
2,3,Patricia Galloway,jamesshawn@martin-kelly.com,2.010269e+11,"283 Steven Groves, Lake Mark, WI 07832",1970-03-10
3,4,Renee Morales,robinbradley@edwards.info,2.017962e+11,NaN,1997-04-19
4,5,Michael Carlson,dcarlson@hotmail.com,2.012953e+11,"184 Rodriguez Mews, South Aaron, VA 35023",1988-08-31


Total prepared rows to insert: 100
Connected to MySQL: localhost DB: bank
('CustomerID', 'int', 'NO', 'PRI', None, '')
('FullName', 'varchar(100)', 'YES', '', None, '')
('Email', 'varchar(60)', 'YES', '', None, '')
('Phone', 'varchar(20)', 'YES', '', None, '')
('Address', 'varchar(50)', 'YES', '', None, '')
('Date_Of_Birth', 'date', 'YES', '', None, '')
Batch 1: inserted/updated 100 rows
✅ Done. Total rows affected: 100


In [ ]:
file_path = "accounts_150.xlsx"
table_name = "accounts"
expected_cols = ["AccountID", "CustomerID", "AccountType", "Balance", "Status"]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Map columns (to handle name mismatches)
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}")

df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols

# 3) Type conversions
df["AccountID"] = pd.to_numeric(df["AccountID"], errors="coerce").astype("Int64")
df["CustomerID"] = pd.to_numeric(df["CustomerID"], errors="coerce").astype("Int64")
df["Balance"] = pd.to_numeric(df["Balance"], errors="coerce")

df["AccountType"] = df["AccountType"].apply(lambda x: x[:20] if isinstance(x, str) else x)
df["Status"] = df["Status"].apply(lambda x: x[:20] if isinstance(x, str) else x)

# Replace NaN with None
df = df.replace({np.nan: None})

records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 4) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute(f"DESCRIBE {table_name}")
    print("Table schema (DESCRIBE):")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (AccountID, CustomerID, AccountType, Balance, Status)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
      CustomerID = VALUES(CustomerID),
      AccountType = VALUES(AccountType),
      Balance = VALUES(Balance),
      Status = VALUES(Status)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


Loaded rows: 150
Columns in file: ['AccountID', 'CustomerID', 'AccountType', 'Balance', 'Status']
Total prepared rows to insert: 150
Table schema (DESCRIBE):
('AccountID', 'int', 'NO', 'PRI', None, '')
('CustomerID', 'int', 'YES', 'MUL', None, '')
('AccountType', 'varchar(30)', 'YES', '', None, '')
('Balance', 'decimal(10,0)', 'YES', '', None, '')
('Status', 'varchar(50)', 'YES', '', None, '')
Batch 1: inserted/updated 150 rows
✅ Done. Total rows affected: 150


In [ ]:
file_path = "cards_data.xlsx"
table_name = "cards"
expected_cols = ["CardID", "AccountID", "CardType", "ExpiryDate"]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Map columns
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}")

df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols

# 3) Clean data
df["CardID"] = pd.to_numeric(df["CardID"], errors="coerce").astype("Int64")
df["AccountID"] = pd.to_numeric(df["AccountID"], errors="coerce").astype("Int64")

df["ExpiryDate"] = pd.to_datetime(df["ExpiryDate"], dayfirst=True, errors="coerce")
df["ExpiryDate"] = df["ExpiryDate"].dt.strftime("%Y-%m-%d")
df["ExpiryDate"] = df["ExpiryDate"].where(df["ExpiryDate"].notnull(), None)

df = df.replace({np.nan: None})


df["CardType"] = df["CardType"].apply(lambda x: x[:20] if isinstance(x, str) else x)

# 4) Prepare tuples
records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 5) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute(f"DESCRIBE {table_name}")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (CardID, AccountID, CardType, ExpiryDate)
    VALUES (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
      AccountID = VALUES(AccountID),
      CardType = VALUES(CardType),
      ExpiryDate = VALUES(ExpiryDate)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed with error:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


Loaded rows: 150
Columns in file: ['CardID', 'AccountID', 'CardType', 'ExpiryDate']
Total prepared rows to insert: 150
('CardID', 'int', 'NO', 'PRI', None, '')
('AccountID', 'int', 'YES', 'MUL', None, '')
('CardType', 'varchar(50)', 'YES', '', None, '')
('ExpiryDate', 'date', 'YES', '', None, '')
Batch 1: inserted/updated 150 rows
✅ Done. Total rows affected: 150


In [ ]:
file_path = "customer_services.xlsx"
table_name = "customer_services"
expected_cols = ["CustomerID", "ServiceID", "SubscriptionDate"]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Ensure expected columns exist
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}. Rename them to {expected_cols}")

# 3) Reorder/select the expected columns
df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols  # standardize names

print("Preview (first 5 rows):")
display(df.head())

# 4) Clean and convert types
df["CustomerID"] = pd.to_numeric(df["CustomerID"], errors="coerce").astype("Int64")
df["ServiceID"] = pd.to_numeric(df["ServiceID"], errors="coerce").astype("Int64")

# Format date column
df["SubscriptionDate"] = pd.to_datetime(df["SubscriptionDate"], dayfirst=True, errors="coerce")
df["SubscriptionDate"] = df["SubscriptionDate"].dt.strftime("%Y-%m-%d")
df["SubscriptionDate"] = df["SubscriptionDate"].where(df["SubscriptionDate"].notnull(), None)

# Replace NaN with None
df = df.replace({np.nan: None})

# 5) Prepare data tuples
records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 6) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()
    print("Connected to MySQL:", db_config["host"], "DB:", db_config["database"])

    cursor.execute(f"DESCRIBE {table_name};")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (CustomerID, ServiceID, SubscriptionDate)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE
      ServiceID = VALUES(ServiceID),
      SubscriptionDate = VALUES(SubscriptionDate)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed with error:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


Loaded rows: 282
Columns in file: ['CustomerID', 'ServiceID', 'SubscriptionDate']
Preview (first 5 rows):



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\kira\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


,CustomerID,ServiceID,SubscriptionDate
0,1,4,2023-05-22
1,1,9,2022-10-15
2,2,10,2023-02-09
3,2,9,2024-02-29
4,2,5,2023-08-14


Total prepared rows to insert: 282
Connected to MySQL: localhost DB: bank
('CustomerID', 'int', 'NO', 'PRI', None, '')
('ServiceID', 'int', 'NO', 'PRI', None, '')
('SubscriptionDate', 'date', 'YES', '', None, '')
Batch 1: inserted/updated 282 rows
✅ Done. Total rows affected: 282


C:\Users\kira\AppData\Local\Temp\ipykernel_20416\3101534857.py:54: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["SubscriptionDate"] = pd.to_datetime(df["SubscriptionDate"], dayfirst=True, errors="coerce")


In [ ]:
file_path = "services_list.xlsx"
table_name = "services"
expected_cols = ["ServiceID", "ServiceName", "Description"]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Map columns (to handle name mismatches)
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}")

df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols


# Replace NaN with None
df = df.replace({np.nan: None})

records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 4) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute(f"DESCRIBE {table_name}")
    print("Table schema (DESCRIBE):")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (ServiceID, ServiceName, Description)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE
      ServiceName = VALUES(ServiceName),
      Description = VALUES(Description)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


Loaded rows: 10
Columns in file: ['ServiceID', 'ServiceName', 'Description']
Total prepared rows to insert: 10
Table schema (DESCRIBE):
('ServiceID', 'int', 'NO', 'PRI', None, '')
('ServiceName', 'varchar(50)', 'YES', '', None, '')
('Description', 'varchar(300)', 'YES', '', None, '')
Batch 1: inserted/updated 2 rows
✅ Done. Total rows affected: 2


In [ ]:
file_path = "employees_50.xlsx"
table_name = "employees"
expected_cols = ["EmployeeID", "BranchID", "FullName" ,"Position" , "HireDate" ]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Map columns (to handle name mismatches)
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}")

df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols

# 3) Type conversions
df["EmployeeID"] = pd.to_numeric(df["EmployeeID"], errors="coerce").astype("Int64")
df["BranchID"] = pd.to_numeric(df["BranchID"], errors="coerce").astype("Int64")

# Replace NaN with None
df = df.replace({np.nan: None})

records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 4) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute(f"DESCRIBE {table_name}")
    print("Table schema (DESCRIBE):")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (EmployeeID, BranchID, FullName ,Position , HireDate )
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
      BranchID = VALUES(BranchID),
      FullName = VALUES(FullName),
      Position = VALUES(Position),
      HireDate = VALUES(HireDate)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


Loaded rows: 50
Columns in file: ['EmployeeID', 'BranchID', 'FullName', 'Position', 'HireDate']
Total prepared rows to insert: 50
Table schema (DESCRIBE):
('EmployeeID', 'int', 'NO', 'PRI', None, '')
('BranchID', 'int', 'YES', 'MUL', None, '')
('FullName', 'varchar(50)', 'YES', '', None, '')
('Position', 'varchar(50)', 'YES', '', None, '')
('HireDate', 'date', 'YES', '', None, '')
Batch 1: inserted/updated 100 rows
✅ Done. Total rows affected: 100


In [ ]:
file_path = "employee_customer_200.xlsx"
table_name = "employees_customers"
expected_cols = ["EmployeeID", "CustomerID"]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Map columns (to handle name mismatches)
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}")

df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols

# 3) Type conversions
df["EmployeeID"] = pd.to_numeric(df["EmployeeID"], errors="coerce").astype("Int64")
df["CustomerID"] = pd.to_numeric(df["CustomerID"], errors="coerce").astype("Int64")

# Replace NaN with None
df = df.replace({np.nan: None})

records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 4) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute(f"DESCRIBE {table_name}")
    print("Table schema (DESCRIBE):")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (EmployeeID, CustomerID)
    VALUES (%s, %s)
    ON DUPLICATE KEY UPDATE
      EmployeeID = VALUES(EmployeeID),
      CustomerID = VALUES(CustomerID)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass


Loaded rows: 200
Columns in file: ['EmployeeID', 'CustomerID', 'RelationshipType', 'StartDate']
Total prepared rows to insert: 200
Table schema (DESCRIBE):
('EmployeeID', 'int', 'NO', 'PRI', None, '')
('CustomerID', 'int', 'NO', 'PRI', None, '')
Batch 1: inserted/updated 199 rows
✅ Done. Total rows affected: 199


In [ ]:
file_path = "loans_data_formatted.xlsx"
table_name = "loans"
expected_cols = ["LoanID", "CustomerID", "LoanType", "Amount" , "Status", "StartDate" ,"EndDate" ]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Map columns (to handle name mismatches)
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}")

df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols

# 3) Type conversions
df["StartDate"] = pd.to_datetime(df["StartDate"], dayfirst=True, errors="coerce")
df["StartDate"] = df["StartDate"].dt.strftime("%Y-%m-%d")
df["EndDate"] = pd.to_datetime(df["EndDate"], dayfirst=True, errors="coerce")
df["EndDate"] = df["EndDate"].dt.strftime("%Y-%m-%d")


# Replace NaN with None
df = df.replace({np.nan: None})

records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 4) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute(f"DESCRIBE {table_name}")
    print("Table schema (DESCRIBE):")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (LoanID, CustomerID, LoanType, Amount , Status, StartDate ,EndDate)
    VALUES (%s, %s, %s, %s,%s,%s,%s)
    ON DUPLICATE KEY UPDATE
      CustomerID = VALUES(CustomerID),
      LoanType = VALUES(LoanType),
      Amount = VALUES(Amount),
      Status = VALUES(Status),
      StartDate = VALUES(StartDate),
      EndDate = VALUES(EndDate)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i : i + batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass

Loaded rows: 200
Columns in file: ['LoanID', 'CustomerID', 'LoanType', 'Amount', 'Status', 'StartDate', 'EndDate']
Total prepared rows to insert: 200
Table schema (DESCRIBE):
('LoanID', 'int', 'NO', 'PRI', None, '')
('CustomerID', 'int', 'YES', 'MUL', None, '')
('LoanType', 'varchar(50)', 'YES', '', None, '')
('Amount', 'decimal(10,0)', 'YES', '', None, '')
('Status', 'varchar(50)', 'YES', '', None, '')
('StartDate', 'date', 'YES', '', None, '')
('EndDate', 'date', 'YES', '', None, '')
Batch 1: inserted/updated 200 rows
✅ Done. Total rows affected: 200


In [ ]:
file_path = "loan_payments.xlsx"
table_name = "loanpayments"
expected_cols = ["PaymentID", "LoanID", "PaymentDate", "AmountPaid"]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Map columns (to handle name mismatches)
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}")

df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols

# 3) Type conversions
df["PaymentID"] = pd.to_numeric(df["PaymentID"], errors="coerce").astype("Int64")
df["LoanID"] = pd.to_numeric(df["LoanID"], errors="coerce").astype("Int64")
df["PaymentDate"] = pd.to_datetime(df["PaymentDate"], dayfirst=True, errors="coerce")
df["PaymentDate"] = df["PaymentDate"].dt.strftime("%Y-%m-%d")

# Replace NaN with None
df = df.replace({np.nan: None})

records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 4) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute(f"DESCRIBE {table_name}")
    print("Table schema (DESCRIBE):")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (PaymentID, LoanID, PaymentDate, AmountPaid)
    VALUES (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
      LoanID = VALUES(LoanID),
      PaymentDate = VALUES(PaymentDate),
      AmountPaid = VALUES(AmountPaid)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i : i + batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass

Loaded rows: 1294
Columns in file: ['PaymentID', 'LoanID', 'PaymentDate', 'AmountPaid']
Total prepared rows to insert: 1294
Table schema (DESCRIBE):
('PaymentID', 'int', 'NO', 'PRI', None, '')
('LoanID', 'int', 'YES', 'MUL', None, '')
('PaymentDate', 'date', 'YES', '', None, '')
('AmountPaid', 'decimal(10,0)', 'YES', '', None, '')
Batch 1: inserted/updated 500 rows
Batch 2: inserted/updated 500 rows
Batch 3: inserted/updated 294 rows
✅ Done. Total rows affected: 1294


In [ ]:
file_path = "transactions_500.xlsx"
table_name = "transactions"
expected_cols = ["TransactionID", "AccountID", "TransactionDate", "Amount","TransactionType"]
batch_size = 500

# 1) Read Excel
df = pd.read_excel(file_path, engine="openpyxl")
print("Loaded rows:", len(df))
print("Columns in file:", list(df.columns))

# 2) Map columns (to handle name mismatches)
cols_lower = {c.lower(): c for c in df.columns}
mapped = {}
for col in expected_cols:
    if col in df.columns:
        mapped[col] = col
    elif col.lower() in cols_lower:
        mapped[col] = cols_lower[col.lower()]
    else:
        mapped[col] = None

missing = [c for c, v in mapped.items() if v is None]
if missing:
    raise SystemExit(f"Missing columns in Excel: {missing}")

df = df[[mapped[c] for c in expected_cols]]
df.columns = expected_cols

# 3) Type conversions
df["TransactionID"] = pd.to_numeric(df["TransactionID"], errors="coerce").astype("Int64")
df["AccountID"] = pd.to_numeric(df["AccountID"], errors="coerce").astype("Int64")


# Replace NaN with None
df = df.replace({np.nan: None})

records = [tuple(x) for x in df[expected_cols].to_numpy()]
print("Total prepared rows to insert:", len(records))

# 4) Insert into MySQL
try:
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute(f"DESCRIBE {table_name}")
    print("Table schema (DESCRIBE):")
    for row in cursor.fetchall():
        print(row)

    insert_sql = f"""
    INSERT INTO {table_name} (TransactionID, AccountID, TransactionDate, Amount,TransactionType)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
      AccountID = VALUES(AccountID),
      TransactionDate = VALUES(TransactionDate),
      Amount = VALUES(Amount),
      TransactionType = VALUES(TransactionType)
    """

    total = 0
    for i in range(0, len(records), batch_size):
        chunk = records[i : i + batch_size]
        try:
            cursor.executemany(insert_sql, chunk)
            conn.commit()
            total += cursor.rowcount
            print(f"Batch {i//batch_size + 1}: inserted/updated {cursor.rowcount} rows")
        except Error as e_batch:
            conn.rollback()
            print("Batch failed:", e_batch)

    print("✅ Done. Total rows affected:", total)

except Exception as e:
    print("Fatal error:", e)
finally:
    try:
        cursor.close()
        conn.close()
    except:
        pass

Loaded rows: 500
Columns in file: ['TransactionID', 'AccountID', 'TransactionDate', 'Amount', 'TransactionType']
Total prepared rows to insert: 500
Table schema (DESCRIBE):
('TransactionID', 'int', 'NO', 'PRI', None, '')
('AccountID', 'int', 'YES', 'MUL', None, '')
('TransactionDate', 'date', 'YES', '', None, '')
('Amount', 'decimal(10,0)', 'YES', '', None, '')
('TransactionType', 'varchar(50)', 'YES', '', None, '')
Batch 1: inserted/updated 500 rows
✅ Done. Total rows affected: 500


C:\Users\kira\AppData\Local\Temp\ipykernel_20416\4081407170.py:45: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["TransactionDate"] = pd.to_datetime(df["TransactionDate"], dayfirst=True, errors="coerce")
